In [3]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook



In [4]:
with open('/Users/12103/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [5]:
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [6]:
# set our API call parameters 
LOCATION = 'Austin,TX'
TERM = 'barbeque'

In [8]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_Austin_BBQ.json"
JSON_FILE

'Data/results_in_progress_Austin_BBQ.json'

In [9]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_Austin_BBQ.json not found. Saving empty list to file.


In [10]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')


- 0 previous results found.


In [11]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [12]:
## How many results total?
total_results = results['total']
total_results

2300

In [13]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [14]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

115

In [15]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [16]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/115 [00:00<?, ?it/s]

In [17]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)

  0%|          | 0/115 [00:00<?, ?it/s]

HTTPError: 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=Austin%2CTX&term=barbeque&offset=1000

In [18]:
## delete file and confirm it no longer exits.
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)

False

In [19]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  



In [20]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages



[i] Data/results_in_progress_Austin_BBQ.json not found. Saving empty list to new file.
- 0 previous results found.


115

In [21]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)


  0%|          | 0/115 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [22]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,YZs1gNSh_sN8JmN_nrpxeA,terry-blacks-barbecue-austin,Terry Black's Barbecue,https://s3-media2.fl.yelpcdn.com/bphoto/jUh-UQ...,False,https://www.yelp.com/biz/terry-blacks-barbecue...,6542,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.5,"{'latitude': 30.2596915716907, 'longitude': -9...",[delivery],$$,"{'address1': '1003 Barton Springs Rd', 'addres...",+15123945899,(512) 394-5899,5063.210482
1,MGzro82Fi4LYvc86acoONQ,franklin-barbecue-austin,Franklin Barbecue,https://s3-media1.fl.yelpcdn.com/bphoto/MsXhN-...,False,https://www.yelp.com/biz/franklin-barbecue-aus...,5925,"[{'alias': 'bbq', 'title': 'Barbeque'}]",4.5,"{'latitude': 30.2701348, 'longitude': -97.7313...",[delivery],$$,"{'address1': '900 E 11th St', 'address2': '', ...",+15126531187,(512) 653-1187,4645.277697
2,EPkf96UcG2W3ttlQphWUGQ,la-barbecue-austin-3,la Barbecue,https://s3-media2.fl.yelpcdn.com/bphoto/ajFK7q...,False,https://www.yelp.com/biz/la-barbecue-austin-3?...,2891,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.5,"{'latitude': 30.254551688240156, 'longitude': ...",[delivery],$$,"{'address1': '2401 E Cesar Chavez', 'address2'...",+15126059696,(512) 605-9696,6820.435333
3,qLJUDPOamhLdbtfbcWyHnA,the-salt-lick-bbq-driftwood,The Salt Lick BBQ,https://s3-media4.fl.yelpcdn.com/bphoto/4qv5K5...,False,https://www.yelp.com/biz/the-salt-lick-bbq-dri...,5424,"[{'alias': 'bbq', 'title': 'Barbeque'}]",4.0,"{'latitude': 30.131736, 'longitude': -98.013335}",[],$$,"{'address1': '18300 Fm 1826', 'address2': '', ...",+15128584959,(512) 858-4959,31226.187448
4,-zfm3zSoMNkLXyBZqseczw,interstellar-bbq-austin,Interstellar BBQ,https://s3-media4.fl.yelpcdn.com/bphoto/NvLrBR...,False,https://www.yelp.com/biz/interstellar-bbq-aust...,505,"[{'alias': 'bbq', 'title': 'Barbeque'}]",4.5,"{'latitude': 30.4618814407535, 'longitude': -9...",[delivery],$$,"{'address1': '12233 Ranch Rd 620 N', 'address2...",+15123826248,(512) 382-6248,18265.694211


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,NuxwiVWnY9l8LnXofbB7LQ,libby-s-lunchbox-cedar-park,Libby’s Lunchbox,https://s3-media4.fl.yelpcdn.com/bphoto/yil_0M...,False,https://www.yelp.com/biz/libby-s-lunchbox-ceda...,3,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...",5.0,"{'latitude': 30.544825394680224, 'longitude': ...",[],NaN,"{'address1': '5600 183A', 'address2': None, 'a...",,,27218.206544
996,sInwcW7OUAvUaNDY1Y7w9g,brew-market-and-cafe-austin,Brew Market & Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/r6-2rX...,False,https://www.yelp.com/biz/brew-market-and-cafe-...,6,"[{'alias': 'beer_and_wine', 'title': 'Beer, Wi...",4.5,"{'latitude': 30.2889345304199, 'longitude': -9...",[],$$,"{'address1': '3842 Airport Blvd', 'address2': ...",+15124721277,(512) 472-1277,5126.141575
997,1rvtRi1PjAikHuIWcK4PkA,7-eleven-austin-65,7-Eleven,https://s3-media1.fl.yelpcdn.com/bphoto/unFKUI...,False,https://www.yelp.com/biz/7-eleven-austin-65?ad...,4,"[{'alias': 'convenience', 'title': 'Convenienc...",2.5,"{'latitude': 30.332840168709, 'longitude': -97...","[pickup, delivery]",$,"{'address1': '7114 N Interstate 35', 'address2...",+15124202266,(512) 420-2266,5897.567863
998,_H1abGPP1ckVIkCgaEXFKw,bb-q-chicken-austin-cedar-park-austin,bb.q Chicken - Austin Cedar Park,https://s3-media2.fl.yelpcdn.com/bphoto/4Y6E-i...,False,https://www.yelp.com/biz/bb-q-chicken-austin-c...,30,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.0,"{'latitude': 30.479647453589585, 'longitude': ...","[pickup, delivery]",NaN,"{'address1': '11301 Lakeline Blvd Ste', 'addre...",+15129920227,(512) 992-0227,19819.819169
999,CtEGObFKWrjoCw5yh3Nj5w,hopdoddy-burger-bar-austin-5,Hopdoddy Burger Bar,https://s3-media3.fl.yelpcdn.com/bphoto/OaVQh5...,False,https://www.yelp.com/biz/hopdoddy-burger-bar-a...,108,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.0,"{'latitude': 30.263778687978153, 'longitude': ...","[pickup, delivery]",$$,"{'address1': '1011 E 5th St', 'address2': 'Ste...",+15128287855,(512) 828-7855,5213.615431


In [23]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

0

In [24]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [25]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_Austin_BBQ.csv.gz', compression='gzip',index=False)
